<a href="https://colab.research.google.com/github/jhuarancca/ASU_DataProcessing/blob/main/assigment03_CreatePartitioningInPostgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root1 WITH SUPERUSER PASSWORD '123456'"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @postgres'

In [ ]:
#!/usr/bin/python3
#
# Interface for the assignement
#
import psycopg2


def getOpenConnection(user='postgres', password='1234', dbname='postgres'):
    conn = None
    try:
        # read connection parameters
        # params = config()

        # connect to the postgresql server
        print('connecting to the postgresql database...')
        # conn = psycopg2.connect(**params)
        conn = psycopg2.connect(
            "dbname='" + dbname + "' user='" + user + "' host='localhost' password='" + password + "'")

        # create a cursor
        cur = conn.cursor()

        # execute a statement
        print('postgresql database version:')
        cur.execute('select version()')

        # display the postgresql database server version
        db_version = cur.fetchone()
        print(db_version)

        # close the communication with the postgresql
        cur.close()
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        # finally:
        #    if conn is not None:
        #        conn.close()
        #        print('Database connection closed.')


def loadRatings(ratingstablename, ratingsfilepath, openconnection):
    # def loadRatings():
    print('loading ratings...')

    cursor = openconnection.cursor()

    lsSql = "drop table if exists " + ratingstablename
    # print(lsSql)
    cursor.execute(lsSql)

    lsSql = "create table " + ratingstablename + " (rowID serial primary key, userID Int, t1 char(5), movieID Int , t3 char(5),  rating real, t5 char(5), timestamp Int)"
    cursor.execute(lsSql)

    loadout = open(ratingsfilepath, 'r')
    cursor.copy_from(loadout, ratingstablename, sep=':',
                     columns=('userID', 't1', 'movieID', 't3', 'rating', 't5', 'timestamp'))

    lsSql = "alter table " + ratingstablename + " drop column t1,drop column  t3,drop column  t5"
    cursor.execute(lsSql)

    cursor.execute("select * from ratings")
    print(cursor.fetchall())
    openconnection.commit()

    cursor.close()
    print('ratings loaded...')


def rangePartition(ratingstablename, numberofpartitions, openconnection):
    print('Range partitioning...')
    cursor = openconnection.cursor()
    maxRating = 5.0  # Videos was ranked from 1 to 5
    ratingRanges = maxRating / numberofpartitions
    partitionNumber = 0
    partitionName = 'range_part'
    ratingStart = 0
    #print(ratingRanges)
    while ratingStart < 5.0:
        lsSql = "drop table if exists " + partitionName + str(partitionNumber)
        cursor.execute(lsSql)
        #print(partitionNumber)
        #print(partitionNumber)
        if ratingStart == 0:
            lsSql = "create table " + partitionName + str(
                partitionNumber) + " as select userID,movieID,rating from " + ratingstablename + " where rating<=" + str(
                ratingStart + ratingRanges) + ";"
            cursor.execute(lsSql)
        else:
            lsSql = "create table " + partitionName + str(
                partitionNumber) + " as select userID,movieID,rating from " + ratingstablename + " where rating>" + str(
                ratingStart ) + " and rating<=" + str(ratingStart + ratingRanges) + ";"
            cursor.execute(lsSql)
        partitionNumber = partitionNumber + 1
        ratingStart = ratingStart + ratingRanges

    print('Range partitioned 01')
    cursor.execute("select * from range_part0")
    print(cursor.fetchall())
    #print('Range partitioned 02')
    #cursor.execute("select * from range_part1")
    print(cursor.fetchall())

    openconnection.commit()
    cursor.close()

    print('Range partitioned...')

def roundRobinPartition(ratingstablename, numberofpartitions, openconnection):
    partitionName = 'rrobin_part'
    print('Round Robin partitioning...')
    cursor = openconnection.cursor()

    lsSql = "select * from " + ratingstablename
    cursor.execute(lsSql)
    rows = cursor.fetchall()

    tablePartition = 0
    while tablePartition < numberofpartitions:
        newTableName = partitionName + str(tablePartition)
        lsSql = "drop table if exists " + partitionName + str(tablePartition)
        cursor.execute(lsSql)

        cursor.execute("create table if not exists %s(userID Int, movieID Int, rating REAL)" % (newTableName))
        tablePartition += 1;

    tablePartition = 0
    for row in rows:
        newTableName =  partitionName + str(tablePartition)
        cursor.execute("insert into %s(userID, movieID, rating) values(%d, %d, %f)" % (newTableName, row[0], row[1], row[2]))
        tablePartition = (tablePartition + 1) % numberofpartitions


    cursor.execute("select * from rrobin_part0")
    print(cursor.fetchall())
    openconnection.commit()
    cursor.close()

    print('Round Robin partitioned...')



def roundRobinPartitionRowID(ratingstablename, numberofpartitions, openconnection):
    partitionName = 'rrobin_part'
    print('Round Robin partitioning...')
    cursor = openconnection.cursor()

    for partitionNumber in range(numberofpartitions):
        lsSql = "drop table if exists " + partitionName + str(partitionNumber)
        cursor.execute(lsSql)

        lsSql = "create table " + partitionName + str(
            partitionNumber) + " as select userID,movieID,rating from " + ratingstablename + " where rowID % " + str(
            numberofpartitions) + " = " + str((partitionNumber + 1) % numberofpartitions) + ";"
        # print(lsSql)
        cursor.execute(lsSql)

    cursor.execute("select * from rrobin_part0")
    print(cursor.fetchall())
    openconnection.commit()
    cursor.close()

    print('Round Robin partitioned...')


def getPartitionRange(rating, openconnection):
    partitionName = 'range_part'
    maxRating = 5.0  # Videos was ranked from 1 to 5
    cursor = openconnection.cursor()

    lsSql = "select table_name from information_schema.tables where table_schema = 'public' and table_name like '" + partitionName + "%'"
    cursor.execute(lsSql)

    partitionNumbersT = 0
    for row in cursor:
        partitionNumbersT = partitionNumbersT + 1
    ratingRange = maxRating / partitionNumbersT

    partitionNumber = 0
    ratingEnd = ratingRange

    while ratingEnd <= 5.0:
        if rating <= ratingEnd:
            break
        ratingEnd = ratingEnd + ratingRange
        partitionNumber = partitionNumber + 1
    return partitionNumber


def rangeinsert(ratingstablename, userid, itemid, rating, openconnection):
    print('Range inserting...')
    partitionName = 'range_part'
    cursor = openconnection.cursor()

    partitionNumber = getPartitionRange(rating, openconnection)
    lsSql = "insert into " + partitionName + str(partitionNumber) + " (userID, movieID, rating) values (" + str(
        userid) + "," + str(itemid) + "," + str(rating) + ")"
    cursor.execute(lsSql)

    cursor.execute("select * from range_part" + str(partitionNumber))
    print(cursor.fetchall())
    openconnection.commit()
    cursor.close()

    print('Range inserted in - ' + partitionName + str(partitionNumber))


def getPartitionRoundRobin(openconnection):
    partitionName = 'rrobin_part'
    cursor = openconnection.cursor()

    lsSql = "select table_name from information_schema.tables where table_schema = 'public' and table_name like '" + partitionName + "%'"
    cursor.execute(lsSql)

    partitionNumbersT = 0
    for row in cursor:
        partitionNumbersT = partitionNumbersT + 1

    endingPartNumber = 0
    maxRowPart = 0
    for tableNumber in range(partitionNumbersT):
        lsSql = "select count(*) from " + partitionName + str(tableNumber) + ";"
        cursor.execute(lsSql)
        currPartCount = cursor.fetchone()[0]

        if currPartCount >= maxRowPart:
            endingPartNumber = tableNumber
            maxRowPart = currPartCount

    currentPartition = (endingPartNumber + 1) % partitionNumbersT

    return currentPartition


def roundrobininsert(ratingstablename, userid, itemid, rating, openconnection):
    print('Round Robin inserting...')
    partitionName = 'rrobin_part'
    cursor = openconnection.cursor()

    currentPartition = getPartitionRoundRobin(openconnection)

    lsSql = "insert into " + partitionName + str(currentPartition) + " (userID, movieID, rating) values (" + str(
        userid) + "," + str(itemid) + "," + str(rating) + ")"
    cursor.execute(lsSql)

    cursor.execute("select * from rrobin_part" + str(currentPartition))
    print(cursor.fetchall())
    openconnection.commit()
    cursor.close()

    print('Round Robin inserted in - ' + partitionName + str(currentPartition))


def createDB(dbname='dds_assignment'):
    """
    We create a DB by connecting to the default user and database of Postgres
    The function first checks if an existing database exists for a given name, else creates it.
    :return:None
    """
    # Connect to the default database
    con = getOpenConnection(dbname='postgres')
    con.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = con.cursor()

    # Check if an existing database with the same name exists
    cur.execute('SELECT COUNT(*) FROM pg_catalog.pg_database WHERE datname=\'%s\'' % (dbname,))
    count = cur.fetchone()[0]
    if count == 0:
        cur.execute('CREATE DATABASE %s' % (dbname,))  # Create the database
    else:
        print('A database named {0} already exists'.format(dbname))

    # Clean up
    cur.close()
    con.close()


def deletepartitionsandexit(openconnection):
    cur = openconnection.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
    l = []
    for row in cur:
        l.append(row[0])
    for tablename in l:
        cur.execute("drop table if exists {0} CASCADE".format(tablename))

    cur.close()


def deleteTables(ratingstablename, openconnection):
    try:
        cursor = openconnection.cursor()
        if ratingstablename.upper() == 'ALL':
            cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
            tables = cursor.fetchall()
            for table_name in tables:
                cursor.execute('DROP TABLE %s CASCADE' % (table_name[0]))
        else:
            cursor.execute('DROP TABLE %s CASCADE' % (ratingstablename))
        openconnection.commit()
    except psycopg2.DatabaseError as e:
        if openconnection:
            openconnection.rollback()
        print('Error %s' % e)
    except IOError as e:
        if openconnection:
            openconnection.rollback()
        print('Error %s' % e)
    finally:
        if cursor:
            cursor.close()


if __name__ == '__main__':
    print('Hello')
    conn = getOpenConnection()
    createDB()
    # loadRatings('ratings', 'ratings.dat', conn)
    loadRatings('ratings', 'test_data.txt', conn)
    #rangePartition('ratings', 2, conn)
    roundRobinPartition('ratings',10, conn)
    # roundrobininsert('ratings', 1, 199, 3.75, conn)
    # rangeinsert('ratings', 1, 188, 3.5, conn)
    deleteTables('ratings', conn)
    deletepartitionsandexit(conn)



/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0,1,2,3
0,1,122,5.0,8.389850e+08
1,1,185,5.0,8.389835e+08
2,1,231,5.0,8.389834e+08
3,1,292,5.0,8.389834e+08
4,1,316,5.0,8.389834e+08
...,...,...,...,...
6898297,49247,2148,3.0,1.107459e+09
6898298,49247,2160,3.5,1.107465e+09
6898299,49247,2161,3.5,1.107465e+09
6898300,49247,2167,1.5,1.107465e+09


In [ ]:


%sql create table ratings(userID int, movieID int, rating float);



 * postgresql+psycopg2://@/postgres
(psycopg2.errors.DuplicateTable) relation "ratings" already exists

[SQL: create table ratings(userID int, movieID int, rating float);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [ ]:
dfRatings.iloc[:, 0:3]


,0,1,2
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0
3,1,292,5.0
4,1,316,5.0
...,...,...,...
6898297,49247,2148,3.0
6898298,49247,2160,3.5
6898299,49247,2161,3.5
6898300,49247,2167,1.5
